In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
movies = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [6]:
movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [7]:
ratings.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [8]:
links.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [9]:
tags.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [10]:
tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [11]:
tmp1 = sum(ratings.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

## Part 1: Spark SQL and OLAP

### Q1: The number of Users

In [14]:
tmp1 = ratings.groupBy("userID").count().toPandas().count()
print('The number of users is {}'.format(tmp1[1]))

The number of users is 610

### Q2: The number of Movies

In [16]:
tmp2 = ratings.groupBy("movieId").count().toPandas().count()
print('The number of movies is {}'.format(tmp2[1]))

The number of movies is 9724

### Q3:  How many movies are rated by users? List movies not rated before

In [18]:
tmp3 = sum(ratings.groupBy("movieId").count().toPandas()['count'] >= 1)
print('The number of rated movies is {}'.format(tmp3))
tmp3_1 = sum(ratings.groupBy("movieId").count().toPandas()['count'] == 0)
print('The number of rated movies is {}'.format(tmp3_1))

The number of rated movies is 9724
The number of rated movies is 0

### Q4: List Movie Genres

In [20]:
tmp4 = movies.groupBy("genres").count().toPandas()
print(format(tmp4))

genres count
0 Comedy|Horror|Thriller 17
1 Adventure|Sci-Fi|Thriller 4
2 Action|Adventure|Drama|Fantasy 6
3 Action|Drama|Horror 1
4 Action|Animation|Comedy|Sci-Fi 2
5 Animation|Children|Drama|Musical|Romance 2
6 Action|Adventure|Drama 27
7 Adventure|Sci-Fi 15
8 Documentary|Musical|IMAX 1
9 Adventure|Children|Fantasy|Sci-Fi|Thriller 1
10 Adventure|Animation 2
11 Musical|Romance|War 1
12 Action|Adventure|Fantasy|Romance 1
13 Adventure|Children|Drama|Fantasy|IMAX 2
14 Comedy|Crime|Horror|Thriller 1
15 Crime|Drama|Fantasy|Horror|Thriller 1
16 Comedy|Mystery|Thriller 3
17 Adventure|Fantasy 13
18 Horror|Romance|Sci-Fi 1
19 Drama|Film-Noir|Romance 2
20 Crime|Drama|Film-Noir|Mystery|Thriller 2
21 Musical|Western 2
22 Adventure|Animation|Children|Musical|Romance 1
23 Comedy|Crime|Mystery|Romance|Thriller 1
24 Action|Animation|Children|Crime 2
25 Adventure|Horror|Sci-Fi 1
26 Comedy|Drama|Romance|Thriller 2
27 Action|Drama|Mystery|Sci-Fi|Thriller 2
28 Adventure|Animation|Children|Comedy|Drama|Fantasy 1
29 Action|Adventure|Mystery|Sci-Fi 1
.. ... ...
921 Adventure|Children|Fantasy 24
922 Drama|Fantasy|Horror|Mystery|Thriller 3
923 Crime|Drama|Film-Noir|Romance|Thriller 1
924 Comedy|Crime|Drama|Mystery 2
925 Action|Adventure|Comedy|Crime|Drama|Film-Noir|... 1
926 Comedy|Drama|Musical|IMAX 1
927 Horror|Sci-Fi|Western 1
928 Adventure|Animation|Fantasy|Sci-Fi 2
929 Crime|Horror|Mystery 1
930 Comedy|Documentary|Drama 2
931 Animation|Fantasy|Sci-Fi|War 1
932 Adventure|Drama|War|Western 2
933 Adventure|Animation|Children|Comedy|Fantasy|IMAX 3
934 Adventure|Children|Comedy|Fantasy|Romance 1
935 Children|Comedy|Romance|Sci-Fi 1
936 Action|Fantasy|Horror 2
937 Adventure|Comedy|Fantasy|Romance|IMAX 1
938 Action|Crime|Drama|Thriller 61
939 Crime|Film-Noir|Thriller 4
940 Children|Fantasy|Musical 3
941 Adventure|Animation|Drama|Horror 1
942 Adventure|Children|Comedy|Musical 2
943 Action|Adventure|Drama|War 22
944 Drama|Horror|Sci-Fi|Thriller 4
945 Animation|Comedy|Fantasy|Musical|Romance 1
946 Action|Adventure|Animation|Comedy|Thriller 1
947 Comedy|Drama 435
948 Drama|Mystery|Romance 9
949 Action|Comedy|Drama|Thriller 2
950 Action|Drama|Sci-Fi|IMAX 2

[951 rows x 2 columns]

### Q5: Movie for Each Category

In [22]:
#what category here? title?
tmp5 = movies.groupBy("title").count().toPandas()
print(format(tmp5))

title count
0 Fair Game (1995) 1
1 If Lucy Fell (1996) 1
2 Three Wishes (1995) 1
3 Heavenly Creatures (1994) 1
4 Snow White and the Seven Dwarfs (1937) 1
5 Night of the Living Dead (1968) 1
6 When We Were Kings (1996) 1
7 Psycho (1960) 1
8 Annie Hall (1977) 1
9 Men in Black (a.k.a. MIB) (1997) 1
10 Odd Couple II, The (1998) 1
11 In the Heat of the Night (1967) 1
12 Elizabeth (1998) 1
13 First Blood (Rambo: First Blood) (1982) 1
14 Problem Child (1990) 1
15 Gulliver's Travels (1939) 1
16 7th Voyage of Sinbad, The (1958) 1
17 Cry in the Dark, A (1988) 1
18 Man Bites Dog (C'est arrivé près de chez vous)... 1
19 One False Move (1992) 1
20 Quills (2000) 1
21 Before Night Falls (2000) 1
22 O Brother, Where Art Thou? (2000) 1
23 Don't Tell Mom the Babysitter's Dead (1991) 1
24 Love Story (1970) 1
25 Captain Corelli's Mandolin (2001) 1
26 Fat City (1972) 1
27 Angel at My Table, An (1990) 1
28 Hello, Dolly! (1969) 1
29 Birth (2004) 1
... ... ...
9707 Eye for an Eye, An (1981) 1
9708 Joe Versus the Volcano (1990) 1
9709 Scooby-Doo (2002) 1
9710 I'm Starting From Three (Ricomincio da Tre) (1... 1
9711 Tupac: Resurrection (2003) 1
9712 Cool World (1992) 1
9713 Munchies (1987) 1
9714 Power of One, The (1992) 1
9715 Carabineers, The (Carabiniers, Les) (1963) 1
9716 Blueberry (2004) 1
9717 Goal! The Dream Begins (Goal!) (2005) 1
9718 Because I Said So (2007) 1
9719 It's a Boy Girl Thing (2006) 1
9720 TMNT (Teenage Mutant Ninja Turtles) (2007) 1
9721 28 Weeks Later (2007) 1
9722 Spiderwick Chronicles, The (2008) 1
9723 Where the Wild Things Are (2009) 1
9724 Lincoln Lawyer, The (2011) 1
9725 Paul (2011) 1
9726 Only God Forgives (2013) 1
9727 Red Flag (2012) 1
9728 Lone Survivor (2013) 1
9729 We Are What We Are (2013) 1
9730 Think Like a Man Too (2014) 1
9731 Zulu (2013) 1
9732 The Madagascar Penguins in a Christmas Caper (... 1
9733 Jurassic World (2015) 1
9734 The Wailing (2016) 1
9735 Sharknado 4: The 4th Awakens (2016) 1
9736 Kong: Skull Island (2017) 1

[9737 rows x 2 columns]

## Part2: Spark ALS based approach for training model
We will use an RDD-based API from [pyspark.mllib](https://spark.apache.org/docs/2.1.1/mllib-collaborative-filtering.html) to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [24]:
from pyspark.mllib.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row

In [25]:
movie_rating = sc.textFile("/FileStore/tables/ratings.csv")

In [26]:
header = movie_rating.take(1)[0]
rating_data = movie_rating.filter(lambda line: line!=header).map(lambda line: line.split(",")).map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]))).cache() 
ratingdata = spark.createDataFrame(rating_data)

In [27]:
# check three rows
# rating_data.take(3)
ratingdata.show(20,False)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
1 |4.0 |1 |
3 |4.0 |1 |
6 |4.0 |1 |
47 |5.0 |1 |
50 |5.0 |1 |
70 |3.0 |1 |
101 |5.0 |1 |
110 |4.0 |1 |
151 |5.0 |1 |
157 |5.0 |1 |
163 |5.0 |1 |
216 |5.0 |1 |
223 |3.0 |1 |
231 |5.0 |1 |
235 |4.0 |1 |
260 |5.0 |1 |
296 |3.0 |1 |
316 |3.0 |1 |
333 |5.0 |1 |
349 |4.0 |1 |
+-------+------+------+
only showing top 20 rows

Now we split the data into training/validation/testing sets using a 6/2/2 ratio.

In [29]:
# train, validation, test = rating_data.randomSplit([6,2,2],seed = 7856)
(train, validation, test) = ratingdata.randomSplit([0.6,0.2,0.2])
train.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 1| 0.5| 76|
 1| 2.0| 132|
 1| 2.0| 153|
 1| 2.0| 298|
 1| 2.5| 15|
 1| 2.5| 68|
 1| 2.5| 82|
 1| 3.0| 32|
 1| 3.0| 33|
 1| 3.0| 44|
 1| 3.0| 50|
 1| 3.0| 90|
 1| 3.0| 130|
 1| 3.0| 134|
 1| 3.0| 140|
 1| 3.0| 155|
 1| 3.0| 233|
 1| 3.0| 279|
 1| 3.0| 283|
 1| 3.5| 119|
+-------+------+------+
only showing top 20 rows

In [30]:
train.cache()

Out[ 20 ]: DataFrame[movieId: bigint, rating: double, userId: bigint]

In [31]:
validation.cache()

Out[ 21 ]: DataFrame[movieId: bigint, rating: double, userId: bigint]

In [32]:
test.cache()

Out[ 22 ]: DataFrame[movieId: bigint, rating: double, userId: bigint]